In [ ]:
# -*- coding: utf-8 -*-
import os, glob, re, unicodedata
import pandas as pd
from typing import List, Optional

INPUT_DIR   = r"/content/KRUS-data-/dane_excel_kwartalne"
OUTPUT_CSV  = r"./master_records.csv"

# ── nazwy specjalne ──
REGION_NAMES = {"region","województwo","wojewodztwo","woj","kraj","państwo","panstwo"}
PERIOD_NAMES = {"okres","period","kwartał","kwartal","rok","miesiąc","miesiac","year","month", "okres według stanu" }
REGION_PATTERNS = [r"\bwoj(e|ewództw|ewodztw)o", r"\bregion\b", r"\bkraj\b", r"\bpa(ns)?two\b"]
PERIOD_PATTERNS = [
    r"\bokres\b", r"\bperiod\b", r"\bkwarta(ł|l)\b", r"\brok\b", r"\bmiesi(?:ąc|ac)\b",
    r"okres\s+wedlug\s+stanu", r"okres\s+wed[oó]ug\s+stanu"
]

# YYYY-Qn lub YYYYn wariant bez myślnika
PERIOD_TOKEN_RE = re.compile(r"^\d{4}[-/]?q[1-4]$", re.IGNORECASE)

# ── mapa polskich miesięcy ──
MONTHS_MAP = {
    "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
    "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
    "września": "09", "wrzesnia": "09", "października": "10", "pazdziernika": "10",
    "listopada": "11", "grudnia": "12"
}

# Funkcja zamieniająca np. "31 marca 2025 r." → "31.03.2025"
def normalize_polish_dates(text: str) -> str:
    if not isinstance(text, str):
        return text

    def repl(match):
        day = match.group(1).zfill(2)
        month_name = match.group(2).lower()
        year = match.group(3)
        month_num = MONTHS_MAP.get(month_name)
        if not month_num:
            return match.group(0)  # jeśli nieznany miesiąc, zostawiamy oryginał
        return f"{day}.{month_num}.{year}"

    # wzorzec: np. "31 marca 2025 r." albo "7 kwietnia 2023 r.,"
    pattern = r"\b(\d{1,2})\s+(stycznia|lutego|marca|kwietnia|maja|czerwca|lipca|sierpnia|września|wrzesnia|października|pazdziernika|listopada|grudnia)\s+(\d{4})\s*r?\.?,?"
    return re.sub(pattern, repl, text, flags=re.IGNORECASE)

def read_csv_any(path: str) -> pd.DataFrame:
    for enc in ("utf-8-sig","utf-8","cp1250","iso-8859-2"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path)

def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", s or "")
        if unicodedata.category(c) != "Mn"
    )

def norm(s: str) -> str:
    s = strip_accents(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

def find_special_col(columns: List[str], exact_set: set, patterns: List[str]) -> Optional[str]:
    for c in columns:
        if norm(c) in exact_set:
            return c
    for c in columns:
        nc = norm(c)
        if any(re.search(p, nc) for p in patterns):
            return c
    return None

def clean_value(v) -> str:
    """Zamiana NaN -> '', liczby z .0 -> int jako string."""
    if pd.isna(v):
        return ""
    try:
        # liczby całkowite zapisane jako float z .0
        if isinstance(v, float) and v.is_integer():
            return str(int(v))
    except Exception:
        pass
    s = str(v)
    if s.endswith(".0") and s.replace(".0","").isdigit():
        return s.replace(".0","")
    return s

def process_file(path: str) -> pd.DataFrame:
    df = read_csv_any(path)
    df.columns = [c.strip() for c in df.columns]
    dataset = os.path.splitext(os.path.basename(path))[0]

    # specjalne kolumny
    region_col = find_special_col(list(df.columns), REGION_NAMES, REGION_PATTERNS)
    period_col = find_special_col(list(df.columns), PERIOD_NAMES, PERIOD_PATTERNS)
    typ_col    = next((c for c in df.columns if norm(c) == "typ"), None)

    id_vars = []
    if region_col: id_vars.append(region_col)
    if period_col: id_vars.append(period_col)
    if typ_col:    id_vars.append(typ_col)

    value_vars = [c for c in df.columns if c not in id_vars]
    if not value_vars:
        return pd.DataFrame(columns=["dataset","measure","value","region","period","typ"])

    long = df.melt(id_vars=id_vars, value_vars=value_vars,
                   var_name="measure", value_name="value")

    # podstawowe mapowanie
    long["dataset"] = dataset
    long["region"]  = long[region_col] if region_col else ""
    long["period"]  = long[period_col] if period_col else ""
    long["typ"]     = long[typ_col] if typ_col else ""

    # DODATKOWO: jeśli measure wygląda jak 2025-Q1, uzupełnij period
    mask_measure_period = long["measure"].astype(str).str.match(PERIOD_TOKEN_RE)
    long.loc[mask_measure_period & (long["period"] == ""), "period"] = long.loc[mask_measure_period, "measure"]

    # jeśli wartości w kolumnach id_vars wyglądają jak 2025-Q1, uzupełnij period
    for idc in id_vars:
        vals = long[idc].astype(str)
        mask_row_period = vals.str.match(PERIOD_TOKEN_RE)
        long.loc[mask_row_period & (long["period"] == ""), "period"] = vals[mask_row_period]

    # czyszczenie wartości
    for col in ["measure","value","region","period","typ"]:
        long[col] = long[col].map(clean_value)
        long[col] = long[col].map(normalize_polish_dates)  # <<< DODANE TUTAJ

    return long[["dataset","measure","value","region","period","typ"]]

# ── główny przebieg ──
parts: List[pd.DataFrame] = []
for path in sorted(glob.glob(os.path.join(INPUT_DIR, "*.csv"))):
    try:
        part = process_file(path)
        parts.append(part)
        print(f"[OK] {os.path.basename(path)} → {len(part)} rekordów")
    except Exception as e:
        print(f"[BŁĄD] {os.path.basename(path)}: {e}")

if parts:
    master = pd.concat(parts, ignore_index=True)
    master.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"Zapisano: {OUTPUT_CSV} ({len(master)} rekordów)")
else:
    print("Brak danych wejściowych.")
